In [8]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import numpy as np
import pandas as pd
BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy
from backtesting.test import SMA
from backtesting.lib import crossover

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.algofuncs as _af
path = os.getcwd()

In [9]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.buy_price = 0
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if self.buy_price == 0 and crossover(self.ma1, self.ma2):
            self.buy()
            self.buy_price = self.data.Close[-1]
        elif self.buy_price != 0 and crossover(self.ma2, self.ma1):
            self.position.close()
            self.buy_price = 0

In [10]:
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
vn30_ticker = _af.getListVN30()
reports = [['Ticker', 'Avg. Trade [%]', 'Equity Final [$]', 'Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Trades']]
for ticker_id in vn30_ticker:
    ticker = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2018-01-01', '2020-10-05')
    bt = Backtest(ticker, SmaCross, commission=.005, exclusive_orders=False)
    stats = bt.run()
    if stats['_trades'].shape[0]:
        reports = np.append(reports, [[ticker_id, stats['Avg. Trade [%]'], stats['Equity Final [$]'], stats['Return [%]'], stats['Sharpe Ratio'], stats['Win Rate [%]'], stats['_trades'].shape[0]]], axis=0)
ls = pd.DataFrame(data=reports[1:,1:],index=reports[1:,0],columns=reports[0,1:])
ls_s = ls.sort_values(by=['Avg. Trade [%]'], ascending=False, na_position='first')
ls_s

,Avg. Trade [%],Equity Final [$],Return [%],Sharpe Ratio,Win Rate [%],Trades
MWG,1.6989434734762998,16164.403489000011,61.64403489000011,0.7279625946274626,53.333333333333336,15
VCB,1.3259483984090936,12060.73049300002,20.607304930000193,0.3460531680737846,47.05882352941176,17
VPB,1.053391363483036,11735.803815000016,17.35803815000016,0.24665942214731632,38.46153846153847,13
NVL,0.6430133524193637,10974.713419000012,9.747134190000125,0.18988121197378494,37.5,16
EIB,0.6073568277872532,11189.485000000026,11.894850000000261,0.20170336503890784,27.77777777777778,18
DHG,0.5487683563376589,10616.308101500019,6.163081015000189,0.12331695207715285,29.411764705882355,17
CII,0.0515950043090907,10539.040773500015,5.390407735000154,0.11627707117698752,41.66666666666667,12
ROS,-5.395185613290221,4615.14085000001,-53.8485914999999,0.0,14.285714285714285,14
VJC,-2.8981757283847887,5764.744868000012,-42.35255131999988,0.0,42.10526315789473,19
MBB,-2.628582769571186,7335.256623500016,-26.64743376499984,0.0,33.33333333333333,18
